In [1]:
def load_data(debug=False, edgeDetect=False, folder='Fill2NoBg'):
    from keras.utils import to_categorical
    
    # loading data
    PATH = os.getcwd()
    # Define data path
    data_path = PATH + '/../ShapeData/' + folder
    data_dir_list = os.listdir(data_path)
    data_dir_list = [data_dir_list[2]] #0=Ellipse 1=Quadrilateral 2=Triangle
    img_data_list=[]

    for dataset in data_dir_list:
        img_list=os.listdir(data_path+'/'+ dataset)
        print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
        for img in img_list:
            if edgeDetect:
                input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img,1)
                img_data_list.append(edge_detection(input_img))
            else:
                input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img,0)#1=RGB,0=Grey,-1=unchanged
                img_data_list.append(input_img)
#            input_img_resize=cv2.resize(input_img,(IMAGE_HEIGHT,IMAGE_WIDTH))
#            img_data_list.append(input_img_resize)
#             img_data_list.append(edge_detection(input_img))
    
    X_train = np.array(img_data_list)
    
    #Reshaping
    #X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], num_channels))
    
    if debug:
        print('After preprocessing: ')
        print(' - X_train.shape = {}, y_train.shape = {}'.format(X_train.shape, y_train.shape))
        print(' - X_val.shape = {}, y_val.shape = {}'.format(X_val.shape, y_val.shape))
        print(' - X_test.shape = {}, y_test.shape = {}'.format(X_test.shape, y_test.shape))
        print(' - test_images.shape = {}, test_labels.shape = {}'.format(
            test_images.shape, test_labels.shape))  
    
    return X_train

In [2]:
import numpy as np
import os,cv2
X = load_data()
X = X.astype('float32') / 255.0 - 0.5

Using TensorFlow backend.


Loaded the images of dataset-Triangle



In [3]:
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from tensorflow.keras.models import Sequential, Model

def build_autoencoder(img_shape, code_size):
    # The encoder
    encoder = Sequential()
    encoder.add(InputLayer(img_shape))
    encoder.add(Flatten())
    encoder.add(Dense(code_size))

    # The decoder
    decoder = Sequential()
    decoder.add(InputLayer((code_size,)))
    decoder.add(Dense(np.prod(img_shape))) # np.prod(img_shape) is the same as 32*32*3, it's more generic than saying 3072
    decoder.add(Reshape(img_shape))

    return encoder, decoder

In [4]:
IMG_SHAPE = X.shape[1:]
encoder, decoder = build_autoencoder(IMG_SHAPE, 20)

inp = Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = Model(inp,reconstruction)
autoencoder.compile(optimizer='adamax', loss='mse')

print(autoencoder.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 40, 40)]          0         
_________________________________________________________________
sequential (Sequential)      (None, 20)                32020     
_________________________________________________________________
sequential_1 (Sequential)    (None, 40, 40)            33600     
Total params: 65,620
Trainable params: 65,620
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
import tensorflow as tf

# load model
encoder = tf.keras.models.load_model('encoder.h5')
decoder = tf.keras.models.load_model('decoder.h5')

encoder.summary()
decoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               160100    
Total params: 160,100
Trainable params: 160,100
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1600)              161600    
_________________________________________________________________
reshape_1 (Reshape)          (None, 40, 40)            0         
Total params: 161,600
Trainable params: 161,600
Non-trainable params: 0
_________________________________________________________________


In [6]:
import matplotlib.pyplot as plt
def show_image(x):
    plt.imshow(np.clip(x + 0.5, 0, 1),cmap='gray')
show_image(X[6])

In [18]:
img = X[0]
code = encoder.predict(img[None])[0]
reco = decoder.predict(code[None])[0]
show_image(reco)
plt.show()

In [20]:
%matplotlib widget
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

plt.ion()
plt.plot()
def plot_func(val1,val2,val3,val4,val5,val6):
    tmpcode = np.copy(code)
    tmpcode[0] = val1
    tmpcode[1] = val2
    tmpcode[2] = val3
    tmpcode[3] = val4
    tmpcode[4] = val5
    tmpcode[5] = val6
    reco = decoder.predict(tmpcode[None])[0]
    plt.imshow(np.clip(reco + 0.5, 0, 1),cmap='gray')

# interact(plot_func, val1 = widgets.FloatSlider(value=0, min=-2, 
# max=2,step=0.1))
interact(plot_func, 
         val1 = widgets.FloatSlider(min=-2,max=2),
         val2 = widgets.FloatSlider(min=-2,max=2),
         val3 = widgets.FloatSlider(min=-2,max=2),
         val4 = widgets.FloatSlider(min=-2,max=2),
         val5 = widgets.FloatSlider(min=-2,max=2),
         val6 = widgets.FloatSlider(min=-2,max=2),
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='val1', max=2.0, min=-2.0), FloatSlider(value=0.0, de…

<function __main__.plot_func(val1, val2, val3, val4, val5, val6)>

Above proves that PCA(principle component analysis) is required